In [1]:
import numpy as np
import pandas as pd

 #  **Loading DATABASE (Mapped Positions)**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
data = pd.read_excel('/content/drive/MyDrive/ArticleRecognition/Dataset/ArticleExtraction.xlsx', sheet_name='Mapped Positions')

In [4]:
data.sample(5)

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
2776,347271.0,2. 3. 17,Leibung mit Tafelstreifen ca. 20cm,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25925,9992,24900100,ETFUGENBAND,1765.094,...,2.1.4,NaN,NaN,Liefern und montieren von Fenster- und Türleib...,leibung tafelstreifen ca 20 cm,liefern montieren fenster türleibungen tafelst...,leibung tafelstreifen ca 20 cm;liefern montier...,etfugenband,eternitfugenband alu schwarz 25x007 m,etfugenband;eternitfugenband alu schwarz 25x007 m
2911,347582.0,2. 3. 11,Bekleidung Unterschlag Dachüberstand Sporthalle,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25978,9994,11000600,KVH6,18997.530,...,1.1,NaN,NaN,Liefern und montieren von Unterschlagsbekleidu...,bekleidung unterschlag dachüberstand sporthalle,liefern montieren unterschlagsbekleidung faser...,bekleidung unterschlag dachüberstand sporthall...,kvh6,kvh fi / ta nsi c24 din 1052 breite 6 cm,kvh6;kvh fi / ta nsi c24 din 1052 breite 6 cm
315,344596.0,1.4.4.0150,Stahlbetonrohrleitung DN 300 ohne Erdarbeiten,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24930,9970,21310300,STB Rohr DN 300,3990.250,...,4.1,NaN,NaN,"Stahlbetonrohrleitung DN 300 liefern, abladen,...",stahlbetonrohrleitung dn 300 erdarbeiten,stahlbetonrohrleitung dn 300 liefern abladen v...,stahlbetonrohrleitung dn 300 erdarbeiten;stahl...,stb rohr dn 300,stb rohr dn 300 kgm,stb rohr dn 300;stb rohr dn 300 kgm
1731,347617.0,1.001,Baustelle einrichten u. räumen,Long Text 1,"<p>Bürocontainer L 6m B 2,5m aufstellen, räume...",26118,9995,5500-001,LKW 10 to,2517.200,...,3.1.2,NaN,NaN,"Bürocontainer L 6m B 2,5m aufstellen, räumenBü...",baustelle einrichten u räumen,bürocontainer l 6 m b 25 m aufstellen räumenbü...,baustelle einrichten u räumen;bürocontainer l ...,lkw 10 to,lkw 10 to dreiachser,lkw 10 to;lkw 10 to dreiachser
1523,347399.0,2. 2. 3,"Klassenraumfassade, Nordflügel EG, Ostfassade",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25955,9993,26000125,FERMACELL12,4077.141,...,2.3.1,NaN,NaN,Die Elementierung der Wände ist durch den AN z...,klassenraumfassade nordflügel eg ostfassade,elementierung wände wählen mögliche rückverank...,klassenraumfassade nordflügel eg ostfassade;el...,fermacell12,fermacellgipsfaserplatte 125 mm,fermacell12;fermacellgipsfaserplatte 125 mm


In [5]:
df = data.copy()

In [6]:
df

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
0,35340.0,53.38.05.033110,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1742,1103,K141011032000,fugenband,14.3,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Ecke e...,zulage schweißung flache ecke pvcp sfd 24,zulage schweißung flachen ecke innenliegenden ...,zulage schweißung flache ecke pvcp sfd 24;zula...,fugenband,schweißung flache ecke pvcp sfd 24,fugenband;schweißung flache ecke pvcp sfd 24
1,35341.0,53.38.05.033120,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1743,1103,K141011032100,fugenband,22.9,...,1.1,NaN,NaN,Zulage für die Schweißung einer senkrechten Ec...,zulage schweißung senkrechte ecke pvcp sfd 24,zulage schweißung senkrechten ecke innenliegen...,zulage schweißung senkrechte ecke pvcp sfd 24;...,fugenband,schweißgsenkrechte ecke pvcp sfd 24,fugenband;schweißgsenkrechte ecke pvcp sfd 24
2,35342.0,53.38.05.033130,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1744,1103,K141011032200,fugenband,14.7,...,1.1,NaN,NaN,Zulage für die Schweißung eines flachen T-Stüc...,zulage schweißung flaches tstück pvcp sfd 24,zulage schweißung flachen tstücks innenliegend...,zulage schweißung flaches tstück pvcp sfd 24;z...,fugenband,schweißgflaches tstück pvcp sfd 24,fugenband;schweißgflaches tstück pvcp sfd 24
3,35343.0,53.38.05.033140,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1745,1103,K141011032300,fugenband,26.0,...,1.1,NaN,NaN,Zulage für die Schweißung eines senkrechten T-...,zulage schweißung senkrechtes tstück pvcp sfd 24,zulage schweißung senkrechten tstücks innenlie...,zulage schweißung senkrechtes tstück pvcp sfd ...,fugenband,schweißgsenkr tstück pvcp sfd 24,fugenband;schweißgsenkr tstück pvcp sfd 24
4,35344.0,53.38.05.033150,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1746,1103,K141011032400,fugenband,27.1,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Kreuzu...,zulage schweißung flache kreuzung pvcp sfd 24,zulage schweißung flachen kreuzung innenliegen...,zulage schweißung flache kreuzung pvcp sfd 24;...,fugenband,schweißgflache kreuzung pvcp sfd 24,fugenband;schweißgflache kreuzung pvcp sfd 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,344554.0,1.1.1.0010,Baustelleneinrichtung,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24972,9970,MK,NaN,100.0,...,4.3,NaN,NaN,"Einrichten, Unterhalten, Abbauen der Baustelle...",baustelleneinrichtung,einrichten unterhalten abbauen baustelleneinri...,baustelleneinrichtung;einrichten unterhalten a...,NaN,materialcontainer,;materialcontainer
3090,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,4.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenverbau ;großflächenv...,verbau graben,verbau graben 3 m tiefe,verbau graben ;verbau graben 3 m tiefe
3091,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,2.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenver

#**Extracting Useful columns**

In [7]:
df = df[['PositionShortText','LongText','ArticleNumber','Quantity']]

In [8]:
df.head()

,PositionShortText,LongText,ArticleNumber,Quantity
0,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Ecke e...,K141011032000,1.0
1,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer senkrechten Ec...,K141011032100,1.0
2,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Zulage für die Schweißung eines flachen T-Stüc...,K141011032200,1.0
3,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Zulage für die Schweißung eines senkrechten T-...,K141011032300,1.0
4,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Kreuzu...,K141011032400,1.0


In [9]:
df['LongText'][250]

nan

#**Text Preprocessing**

## 1.Handling Missing Data

In [10]:
# Identify missing data
missing_data = df.isnull().any(axis=1)

# Filter out rows with missing data
data_cleaned = df[~missing_data]

# Drop the index associated with the rows containing missing data
data_cleaned = data_cleaned.reset_index(drop=True)

In [11]:
data_cleaned.isnull().sum()

PositionShortText    0
LongText             0
ArticleNumber        0
Quantity             0
dtype: int64

##2.Lowercasing the Text

In [13]:
 # Convert all string columns to lowercase
data_cleaned = data_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

## 3.Remove punctuation and special characters

In [21]:
import re

# Function to remove punctuation and special characters from text
def remove_punctuation_special_characters(text):
    # Define regex pattern to match any character that is not a word character or whitespace
    pattern = r'[^\w\säöüßÄÖÜ]'  # Including German special characters ä, ö, ü, ß, Ä, Ö, and Ü

    # Use regex to substitute all non-word characters and non-whitespace characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Remove punctuation and special characters from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_punctuation_special_characters)

# Remove punctuation and special characters from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_punctuation_special_characters)



## 4.Removing Stopwords

In [15]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords list for German if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Get the set of German stopwords
stop_words_german = set(stopwords.words('german'))

# Function to remove stopwords from text
def remove_stopwords_german(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_german]
    return ' '.join(filtered_tokens)

# Remove stopwords from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_stopwords_german)

# Remove stopwords from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_stopwords_german)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 5.Lemmetization

In [17]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [22]:
!pip install pattern


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 35.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.8/348.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pattern: filename=Pattern-3.6-py3-none-any.whl size=22332702 sha256=b202b2faef6f1dafeafb6c63f10926da6655a7b1ef94d01cb5110dfcdd488f09
  Stored in direc

In [24]:
from pattern.de import lemma

# Function to lemmatize text in German
def lemmatize_text_german(text):
    tokens = text.split()  # Split text into tokens
    lemmatized_tokens = []
    for token in tokens:
        try:
            lemmatized_token = lemma(token)
            lemmatized_tokens.append(lemmatized_token)
        except StopIteration:
            # Handle StopIteration exception
            # You can choose to ignore the token or handle it differently
            pass
    lemmatized_text = ' '.join(lemmatized_tokens)  # Join lemmatized tokens back into text
    return lemmatized_text

# Lemmatize 'LongText' column in German
data_cleaned['LongText'] = data_cleaned['LongText'].apply(lemmatize_text_german)

# Lemmatize 'PositionShortText' column in German
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(lemmatize_text_german)



## 6.Tokenization

In [26]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources for German language if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Tokenize 'LongText' column in German
data_cleaned['LongText_tokens'] = data_cleaned['LongText'].apply(lambda x: word_tokenize(x, language='german'))

# Tokenize 'PositionShortText' column in German
data_cleaned['PositionShortText_tokens'] = data_cleaned['PositionShortText'].apply(lambda x: word_tokenize(x, language='german'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [27]:
data_cleaned


,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zulagen schweißungen flachen ecken pvcpen sfde...,zulagen schweißungen flachen ecken innenliegen...,k141011032000,1.0,"[zulagen, schweißungen, flachen, ecken, innenl...","[zulagen, schweißungen, flachen, ecken, pvcpen..."
1,zulagen schweißungen senkrechen ecken pvcpen s...,zulagen schweißungen senkrechen ecken innenlie...,k141011032100,1.0,"[zulagen, schweißungen, senkrechen, ecken, inn...","[zulagen, schweißungen, senkrechen, ecken, pvc..."
2,zulagen schweißungen flachesen tstücken pvcpen...,zulagen schweißungen flachen tstücksen innenli...,k141011032200,1.0,"[zulagen, schweißungen, flachen, tstücksen, in...","[zulagen, schweißungen, flachesen, tstücken, p..."
3,zulagen schweißungen senkrechtesen tstücken pv...,zulagen schweißungen senkrechen tstücksen inne...,k141011032300,1.0,"[zulagen, schweißungen, senkrechen, tstücksen,...","[zulagen, schweißungen, senkrechtesen, tstücke..."
4,zulagen schweißungen flachen kreuzungen pvcpen...,zulagen schweißungen flachen kreuzungen innenl...,k141011032400,1.0,"[zulagen, schweißungen, flachen, kreuzungen, i...","[zulagen, schweißungen, flachen, kreuzungen, p..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]


In [28]:
# Save the DataFrame to an Excel file
data_cleaned.to_excel('/content/drive/MyDrive/data_cleaned.xlsx', index=False)


#**Train-Test Split of the data**

In [30]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = data_cleaned[['LongText_tokens', 'PositionShortText_tokens','Quantity']]  # Features
y = data_cleaned['ArticleNumber']  # Target variable (replace 'Target_Column' with your actual target column name)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (1606, 3) (1606,)
Test set shape: (402, 3) (402,)


In [40]:
X_train.iloc[0:6]


,LongText_tokens,PositionShortText_tokens,Quantity
1249,"[liefern, montieren, sturzbekleidungen, tafels...","[sturzausbildungen, tafelstreifen, caten, 65cmen]",0.21
261,"[elementierungen, winden, wählen, möglichen, r...","[klassenraumfassaden, nordflügeln, 1en, ogen]",5022.00
607,"[fugen, asphalten, zeichnungen, agen, herstellen]","[fugeasphalten, t12cmben, 5mmen]",1.00
973,"[elementierungen, winden, wählen, möglichen, r...","[klassenraumfassaden, ostflügeln, egen]",6820.00
240,"[liefern, montieren, fenstern, türleibungen, t...","[leibungen, tafelstreifen, caten, 20cmen]",3.00
819,"[liefern, montieren, fassadenbekleidungen, fas...","[bekleidungen, wandflächen, fassadentafeln, fs...",104.40


In [37]:
y_train.head()

1249         24900100
261          40001400
607     k462052003500
973          69910600
240          24100000
Name: ArticleNumber, dtype: object

##Storing each of it as separate variables

In [41]:
# Store X_train and X_test columns as separate variables
X_train_longtext_tokens = X_train['LongText_tokens']
X_train_positionshorttext_tokens = X_train['PositionShortText_tokens']

X_test_longtext_tokens = X_test['LongText_tokens']
X_test_positionshorttext_tokens = X_test['PositionShortText_tokens']

# Store 'Quantity' column for train and test sets
X_train_quantity = X_train['Quantity']
X_test_quantity = X_test['Quantity']

# Store 'ArticleNumber' column for train and test sets
y_train_article_number = y_train.copy()
y_test_article_number = y_test.copy()

